In [18]:
# !pip install ultralytics

In [19]:
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [ ]:
video_path = 'Road_traffic_sample.mp4'
model = YOLO('yolov8x.pt')
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
video_x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_y = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter('output_sample.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (video_x, video_y))

cars_counter = 0
cars_id = set()
entry_frame = {}
exit_frame = {}
cars_speed = {}

while True:
    ret, frame= cap.read()
    if not ret:
        break
    
    h, w, _ = frame.shape
    line = int(h * 0.455)
    cv2.line(frame, (0, line+15), (w, line+65), (0, 255, 0), 2)
    
    b_boxes = model.track(frame, show = False, save = False, persist=True, verbose = False)[0].boxes.data.tolist()
    
    if b_boxes is not None:
      for b_box in b_boxes:
        if len(b_box) < 7:
          break
        
        object_id = int(b_box[4])
        confidence = b_box[5]
        object_class = int(b_box[6])
        
        
        if (confidence < 0.8) or (object_class != 2):
          continue
        
        if object_id not in cars_id:
          cars_id.add(object_id)
          cars_counter += 1
        
        x1, y1, x2, y2 = map(int, b_box[:4])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (223, 70, 0), 2)
        cv2.rectangle(frame, (x1, y1-30), (x2, y1), (223, 70, 0), -1)
        cv2.putText(frame, f'Car {object_id}', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.rectangle(frame, (x1, y2), (x2, y2+30), (223, 70, 0), -1)
        
        if object_id not in entry_frame:
          entry_frame[object_id] = cap.get(cv2.CAP_PROP_POS_FRAMES)
        
        if y1 < line:
          if object_id not in exit_frame:
            exit_frame[object_id] = cap.get(cv2.CAP_PROP_POS_FRAMES)
        
        if object_id not in cars_speed and object_id in exit_frame:
          if (exit_frame[object_id] - entry_frame[object_id]) > 0:
            time_seconds =( exit_frame[object_id] - entry_frame[object_id]) / fps
            cars_speed[object_id] = 3.6 * (15 / (time_seconds))
        
        if object_id in cars_speed:
          cv2.putText(frame, f'{cars_speed[object_id]:.2f} Km/H', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    cv2.rectangle(frame, (50, 50), (330, 130), (0, 255, 0), -1)
    cv2.putText(frame, f'{cars_counter} cars passed', (70, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    
    writer.write(frame)
    frame = cv2.resize(frame, (1000, 600))
    cv2.imshow('Monitor', frame)
    if cv2.waitKey(1)& 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()